**1. Importando librerías**

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.ensemble import GradientBoostingClassifier
import pickle

**2. Análisis exploratorio**

In [ ]:
df = pd.read_csv('../Datasets/heart.csv')

In [ ]:
df.head()

In [ ]:
def describe(df):                        # Funcion para explorar los elementos del dataset
                                         # Ayudará para encontrar valores nulos
    columns=df.columns.to_list()         # Además ayudará a encontrar columnas object y columnas int
    ncol=df.describe().columns.to_list()
    ccol=[]
    for i in columns:
        if(ncol.count(i)==0):
            ccol.append(i)
        else:
            continue
    print('Name of all columns in the dataframe:')
    print(columns)
    print('')
    print('Number of columns in the dataframe:')
    print(len(columns))
    print('')
    print('Name of all numerical columns in the dataframe:')
    print(ncol)
    print('')
    print('Number of numerical columns in the dataframe:')
    print(len(ncol))
    print('')
    print('Name of all categorical columns in the dataframe:')
    print(ccol)
    print('')
    print('Number of categorical columns in the dataframe:')
    print(len(ccol))
    print('')
    print('------------------------------------------------------------------------------------------------')
    print('')
    print('Number of Null Values in Each Column:')
    print('')
    print(df.isnull().sum())
    print('')
    print('')
    print('Number of Unique Values in Each Column:')
    print('')
    print(df.nunique())
    print('')
    print('')
    print('Basic Statistics and Measures for Numerical Columns:')
    print('')
    print(df.describe().T)
    print('')
    print('')
    print('Other Relevant Metadata Regarding the Dataframe:')
    print('')
    print(df.info())
    print('')
    print('')


In [ ]:
describe(df)

In [ ]:
oe=['g','r']
fig = plt.figure(figsize=(15,15))

plt.subplot(3,2,1)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="Sex", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='Sex ', ylabel='Count')

plt.subplot(3,2,2)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="ChestPainType", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='ChestPainType', ylabel='Count')

plt.subplot(3,2,3)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="ExerciseAngina", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='ExerciseAngina', ylabel='Count')

plt.subplot(3,2,4)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="RestingECG", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='RestingECG', ylabel='Count')

plt.subplot(3,2,5)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="ST_Slope", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='ST_Slope', ylabel='Count')

plt.subplot(3,2,6)
plt.style.use('seaborn')
plt.tight_layout()
sns.set_context('talk')
sns.histplot(data=df, x="FastingBS", hue="HeartDisease",multiple="stack",palette=oe)
#ax.set(xlabel='FastingBS', ylabel='Count')

In [ ]:
#Función para visualizar ouliers

def outliers(df_column):
    q75, q25 = np.percentile(df_column, [75 ,25]) 
    iqr = q75 - q25
    print('q75: ',q75)
    print('q25: ',q25)
    print('Inter Quartile Range: ',iqr)
    print('Outliers lie before', q25-1.8*iqr, 'and beyond', q75+1.8*iqr) 

  # Normalmente se usa un IQR de 1.5, pero he usado 1.8 para tener un rango más amplio

    print('Number of Rows with Left Extreme Outliers:', len(df[df_column <q25-1.8*iqr]))
    print('Number of Rows with Right Extreme Outliers:', len(df[df_column>q75+1.8*iqr]))
    plt.tight_layout()
    plt.style.use('seaborn')
    sns.set_context('notebook')
    sns.histplot(data=df, x=df_column, hue="HeartDisease",multiple="stack",palette=oe)

In [ ]:
outliers(df['RestingBP']) 

In [ ]:
# Como los outliers estan ligeramente sesgados hacia el extremo derecho, me los quedo y dropeo los del extremo izquierdo
df=df[df.RestingBP>=84]
len(df)

In [ ]:
df.head()

In [ ]:
outliers(df['Cholesterol']) 
#Es imposible que una persona tenga 0 de colesterol xdddd
#Más adelante lidiaré con este problema

In [ ]:
#De momento me quedo con los valores menores de 500 de colesterol
df=df[df.Cholesterol<=500]
df.head()
len(df)

In [ ]:
df.head()

In [ ]:
outliers(df['MaxHR'])

In [ ]:
outliers(df['Oldpeak'])

**3. Cambiando las cadenas de texto con valores numéricos usando One Hit Encoder**

In [ ]:
def OHE(dfcolumn):
    global df
    dfcolumn.nunique()
    len(df.columns)
    finallencol = (dfcolumn.nunique() - 1) + (len(df.columns)-1)
    dummies = pd.get_dummies(dfcolumn, drop_first=True, prefix=dfcolumn.name)
    df=pd.concat([df,dummies],axis='columns')
    df.drop(columns=dfcolumn.name,axis=1,inplace=True) #Me cargo la columna original
    if(finallencol==len(df.columns)):
        print('Todo correcto') 
        print('')
    else:
        print('Error')
    return df

In [ ]:
OHE(df['ChestPainType'])
OHE(df['Sex'])
OHE(df['RestingECG'])
OHE(df['ExerciseAngina'])
OHE(df['ST_Slope'])

**4. Lidiando con el problema de la gente que tiene el colesterol a 0**

In [ ]:
df[df.Cholesterol==0].mean() #Intento de entender cómo se ven los datos cuando hay valores missing, como el colesterol a 0

In [ ]:
df[df.Cholesterol>0].mean() #Aqui viendo si los que tienen el colesterol a 0 no se tienen en cuenta

In [ ]:
print('Mean: ',df['Cholesterol'].mean())
print('Median: ',df['Cholesterol'].median())

In [ ]:
df[df['Cholesterol']>0].Cholesterol.mean()

In [ ]:
df.describe().T

In [ ]:
# Relleno los valores 0 del colesterol usando KNN-Imputation 
df['Cholesterol'].replace(to_replace = 0, value =np.nan, inplace=True)
KNN_imputed = KNNImputer(n_neighbors=5)
I=KNN_imputed.fit_transform(df)
Cholesterol=[]
for i in range(0,len(df)):
  Cholesterol.append(I[i][2])
df['Cholesterol']=Cholesterol 

In [ ]:
outliers(df['Cholesterol']) 

**5. Transformando los datos para tener mejores resultados**

In [ ]:
# VIF (Variance Inflation Factor) para buscar colinealidad entre variables independientes
vif = df.copy()
vif.drop(columns='HeartDisease',axis=1,inplace=True)
vif_data = pd.DataFrame()
vif_data["feature"] = vif.columns
vif_data["VIF"] = [variance_inflation_factor(vif.values, i) for i in range(len(vif.columns))]

In [ ]:
vif_data # Una buena práctica es trabajar con variables que tengan un VIF >= 10

In [ ]:
# Escalando los datos para mayor eficacia

from sklearn.preprocessing import StandardScaler # #Convirte a standard normal distribtuion
scaler=StandardScaler()
scaler.fit(vif)
scaled_data=scaler.transform(vif)
scaled_data

In [ ]:
from sklearn.decomposition import PCA # Reduciendo dimensiones usando Principal Component Analysis para compensar las variables que tengan un VIF alto
pca=PCA(n_components=11)
pca.fit(scaled_data)
x_pca=pca.transform(scaled_data)
x_pca


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
df["HeartDisease"] = scaler.fit_transform(df["HeartDisease"].values.reshape(-1,1))

In [ ]:
df.head()

**5.1 Haciendo la X y la Y**

In [ ]:
y = df["HeartDisease"]

In [ ]:
x = df.drop(columns='HeartDisease',axis=1, inplace=True)

In [ ]:
df.sample()

**6. Modelo final y algoritmos usados**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

6.1 Support Vector Machine

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state=0)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
print(classification_report(y_test, y_pred))
print('')
print('------------------------')
print('Confusion Matrix')
print('------------------------')
print('')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
print(confusion_matrix(y_test, y_pred))
plot_confusion_matrix(classifier, x_test, y_test,cmap="binary") 
plt.grid(False)
plt.show()

6.2 Logistic Regression Function

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))
print('')
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

6.3 Random Forest Classifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 103, stop = 300, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_pca, df['HeartDisease'], test_size=0.25, random_state=0)
clf=RandomForestClassifier(n_estimators=124,min_samples_split= 2,
                           min_samples_leaf= 1,max_features='sqrt',max_depth=None, bootstrap=False)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
import warnings
warnings.filterwarnings("ignore")
print(classification_report(y_test, y_pred))
print('')
print('------------------------')
print('Confusion Matrix')
print('------------------------')
print('')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
print(confusion_matrix(y_test, y_pred))
plot_confusion_matrix(clf, x_test, y_test,cmap="binary") 
plt.grid(False)
plt.show()

6.4 Support Vector Machine (Radial Basis Function Kernel) with Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000],
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=5)
grid.fit(x_train, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
y_pred = grid.predict(x_test)
print(classification_report(y_test, y_pred))
print('')
print('------------------------')
print('Confusion Matrix')
print('------------------------')
print('')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
print(confusion_matrix(y_test, y_pred))
plot_confusion_matrix(grid, x_test, y_test,cmap="binary") 
plt.grid(False)
plt.show()


6.5 KNN Classifier

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_pca, df['HeartDisease'], test_size=0.25, random_state=0)
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(x_train, y_train)
y_pred=knn.predict(x_test)
print(classification_report(y_test, y_pred))
print('')
print('------------------------')
print('Confusion Matrix')
print('------------------------')
print('')
print(confusion_matrix(y_test, y_pred))
plot_confusion_matrix(knn, x_test, y_test,cmap="binary") 
plt.grid(False)
plt.show()

6.6 Gradient Boosting Algorithm

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
clff = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, max_depth=1, random_state=23)
clff.fit(x_train, y_train)
y_pred=clff.predict(x_test)
print(classification_report(y_test, y_pred))
print('')
print('------------------------')
print('Confusion Matrix')
print('------------------------')
print('')
print(confusion_matrix(y_test, y_pred))
plot_confusion_matrix(clff, x_test, y_test,cmap="binary") 
plt.grid(False)
plt.show()

In [ ]:
## He intentado tb usar Neural Network Using Tensor flow y el Gaussian Navie Bayes Algorithm pero eran demasiado complicados

In [ ]:
filename = 'modelo_final2.sav'
pickle.dump(clff, open(filename, 'wb'))

In [ ]:
a = df.loc[750]
a

In [ ]:
df[df==df.loc[750]].dropna()

In [ ]:
b = df.loc[491]
b

In [ ]:
df[df==df.loc[491]].dropna()

In [ ]:
x_test.sample()

In [ ]:
y_pred = clff.predict(x_test)

In [ ]:
y_pred